In [1]:
from Skiier_revised import *
import pandas as pd
import time
from tqdm import tqdm

### 실제 정답
- state 10,40 -> 똑같음 찾기 어려움

In [8]:
real_solution =np.array([
  [-5.410774, -5.107744],
  [-4.441077, -4.410774],
  [-3.666667, -3.441077],
  [-2.666667, -3.344108],
  [-1.666667, -1.666667],
  [-2.000000, -1.666667],
  [-1.000000, -1.666667],
  [ 0.000000,  0.000000]] )


In [28]:
Q = np.zeros([real_solution.shape[0],real_solution.shape[1]] )
argmax=np.argmax(real_solution, axis=1)
   

for i in range(len(Q)):
    if argmax[i] ==1 :
        Q[i] = [0,1]
    elif argmax[i] ==0 :
        Q[i] = [1,0]
        
print(pd.DataFrame(Q, columns=['n','s'], index =np.arange(0,80,10)).T)

    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0


### Policy iteration 1 - MC Control 

### Grid Search

In [58]:
exploration_decay =np.arange(0.989, 0.9999, 0.0005)
learning_rate = np.arange(0.005, 0.02, 0.001)

In [59]:
learning_rate

array([0.005, 0.006, 0.007, 0.008, 0.009, 0.01 , 0.011, 0.012, 0.013,
       0.014, 0.015, 0.016, 0.017, 0.018, 0.019])

In [51]:
exploration_decay

array([0.989 , 0.9895, 0.99  , 0.9905, 0.991 , 0.9915, 0.992 , 0.9925,
       0.993 , 0.9935, 0.994 , 0.9945, 0.995 , 0.9955, 0.996 , 0.9965,
       0.997 , 0.9975, 0.998 , 0.9985, 0.999 , 0.9995])

In [53]:

for decay in tqdm(exploration_decay)  :
    for lr in learning_rate :
        beg_time =time.time()
        num_ep = 10**5
        q_s_a = q_s_a_init
        pi = pi_50
        exploration_rate = 1

        for epi_i in range(1,num_ep) :
            sample_path_i = simul_path(pi, P_normal, P_speed, R_s_a)
            q_s_a = pol_eval_MC(sample_path_i, q_s_a, alpha = lr)
            pi = pol_imp(pi, q_s_a, exploration_rate)
            exploration_rate *= decay  # exponential decay
    
        end_time =time.time()
        result_q = pd.DataFrame(q_s_a, columns =['n','s'], index= states)
        result_pi = pd.DataFrame(pi, columns =['n','s'], index= states)
        print("Time difference of {} sec".format(end_time- beg_time))
        print("-------------")
        print(result_pi.T)
        print("-------------")
        print(result_q.T)
        print("-------------");
        print("MSE :",((q_s_a-real_solution)**2).mean(), " decaying : ",decay, "learning_rate : ", lr)
        print("\n")
       


  0%|                                                                                           | 0/22 [00:00<?, ?it/s]

Time difference of 22.198956966400146 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0
s  1.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.477877 -4.534605 -3.676550 -2.629453 -1.825229 -1.763590 -1.000000  0.0
s -5.133606 -4.341430 -3.430027 -2.755503 -1.680985 -1.817711 -1.000656  0.0
-------------
MSE : 0.05715065422109222  decaying :  0.989 learning_rate :  0.005


Time difference of 22.714230060577393 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0
s  1.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0
-------------
         0         10        20        30        40        50       60   70
n -5.468926 -4.644617 -3.716892 -2.737651 -1.807508 -1.781806 -1.00000  0.0
s -5.126061 -4.475937 -3.472022 -2.784129 -1.770660 -1.806346 -1.00238  0.0
-------------
MSE : 0.05690838236111528  decaying :  0.989 learni

  5%|███▋                                                                            | 1/22 [06:02<2:06:59, 362.84s/it]

Time difference of 26.58487343788147 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0
s  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.878469 -4.866431 -3.646986 -2.644174 -2.115951 -1.973484 -1.000000  0.0
s -5.250207 -4.845018 -4.061461 -3.082099 -1.644174 -2.115951 -1.040404  0.0
-------------
MSE : 0.11625458948372688  decaying :  0.989 learning_rate :  0.019


Time difference of 23.417613744735718 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.387230 -4.544409 -3.693680 -2.657734 -1.723328 -1.779452 -1.000000  0.0
s -5.120146 -4.443208 -3.494227 -2.766280 -1.788728 -1.723328 -1.002096  0.0
-------------
MSE : 0.05384834393800945  decaying :  0.98949999

  9%|███████▎                                                                        | 2/22 [12:17<2:03:17, 369.90s/it]

...

Time difference of 27.202516555786133 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0
s  1.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.864813 -4.986775 -3.563334 -2.558493 -2.017100 -2.067379 -1.000000  0.0
s -5.280233 -4.825730 -3.946232 -3.101348 -1.558493 -2.017100 -1.629006  0.0
-------------
MSE : 0.08160236284732322  decaying :  0.9984999999999989 learning_rate :  0.019


Time difference of 24.916338205337524 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50       60   70
n -5.448374 -4.563685 -3.666385 -2.689029 -1.656628 -1.811644 -1.00000  0.0
s -5.067489 -4.504809 -3.428894 -2.796549 -1.854313 -1.656628 -1.19499  0.0
-------------
MSE : 0.03879679344377998  decaying : 

 95%|███████████████████████████████████████████████████████████████████████████▍   | 21/22 [2:09:11<06:20, 380.61s/it]

Time difference of 28.1682710647583 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0
s  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.866379 -4.741378 -3.595766 -2.590964 -2.005733 -2.058357 -1.000000  0.0
s -5.291321 -4.866379 -4.154214 -3.611502 -1.590964 -2.005733 -1.792638  0.0
-------------
MSE : 0.08654945356856139  decaying :  0.9989999999999989 learning_rate :  0.019


Time difference of 24.64059805870056 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.492428 -4.579074 -3.774616 -2.630345 -1.781261 -2.009329 -1.000000  0.0
s -5.136024 -4.425083 -3.477238 -3.072377 -1.670640 -1.781261 -1.572316  0.0
-------------
MSE : 0.009381304802374002  decaying :

100%|███████████████████████████████████████████████████████████████████████████████| 22/22 [2:15:31<00:00, 369.59s/it]

Time difference of 26.368605613708496 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0
s  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.785509 -4.785442 -3.797262 -2.780322 -1.972530 -2.027226 -1.000000  0.0
s -5.423814 -4.788589 -3.929935 -3.628461 -1.780322 -1.972530 -1.730235  0.0
-------------
MSE : 0.06601706954444354  decaying :  0.9994999999999988 learning_rate :  0.019




#### 실험 결과 

정답과 같게 나온 조합 

MSE : 0.05384834393800945  decaying :  0.9894999999999999 learning_rate :  0.005

MSE : 0.053464736206858175  decaying :  0.9894999999999999 learning_rate :  0.006

MSE : 0.051484285230378006  decaying :  0.9894999999999999 learning_rate :  0.007

MSE : 0.05754899517040677  decaying :  0.9894999999999999 learning_rate :  0.008

MSE : 0.053950694149495645  decaying :  0.9894999999999999 learning_rate :  0.009000000000000001

MSE : 0.0468714912101883  decaying :  0.9894999999999999 learning_rate :  0.01

MSE : 0.05360575638062484  decaying :  0.9894999999999999 learning_rate :  0.012

MSE : 0.056302162483883055  decaying :  0.9894999999999999 learning_rate :  0.014000000000000002
MSE : 0.05321477240151025  decaying :  0.9899999999999999 learning_rate :  0.005

MSE : 0.06847038978392139  decaying :  0.9899999999999999 learning_rate :  
0.018000000000000002

MSE : 0.053401615771450306  decaying :  0.9904999999999998 learning_rate :  0.005

MSE : 0.0623687014415937  decaying :  0.9909999999999998 learning_rate :  0.013000000000000001

MSE : 0.05201147552850681  decaying :  0.9914999999999997 learning_rate :  0.01

MSE : 0.053999893105639366  decaying :  0.9919999999999997 learning_rate :  0.009000000000000001

MSE : 0.06205376994442717  decaying :  0.9919999999999997 learning_rate :  0.011

MSE : 0.049606493916748054  decaying :  0.9949999999999993 learning_rate :  0.007

MSE : 0.04740661729748284  decaying :  0.9969999999999991 learning_rate :  0.016

MSE : 0.044212770816494995  decaying :  0.997999999999999 learning_rate :  0.008

MSE : 0.03736370689542816  decaying :  0.997999999999999 learning_rate :  0.009000000000000001

MSE : 0.03606687793858906  decaying :  0.9984999999999989 learning_rate :  0.007

MSE : 0.038945670136686594  decaying :  0.9984999999999989 learning_rate :  0.009000000000000001

MSE : 0.012487838559367154  decaying :  0.9989999999999989 learning_rate :  0.009000000000000001

MSE : 0.01317764920460316  decaying :  0.9989999999999989 learning_rate :  0.011

MSE : 0.014124359952948074  decaying :  0.9994999999999988 learning_rate :  0.008

MSE : 0.00734581922269165  decaying :  0.9994999999999988 learning_rate :  0.009000000000000001
.. 다수 



### 파라미터 실험
MSE : 0.00734581922269165  

decaying :  0.9995 learning_rate :  0.009

변동성이 너무 큼

-> 위의 search 결과는 운이 좋아서 그런것으로 보임

In [42]:
beg_time =time.time()
num_ep = 10**5
q_s_a = q_s_a_init
pi = pi_50
exploration_rate = 1

for epi_i in range(1,num_ep) :
    sample_path_i = simul_path(pi, P_normal, P_speed, R_s_a)
    q_s_a = pol_eval_MC(sample_path_i, q_s_a, alpha = 0.009)
    pi = pol_imp(pi, q_s_a, exploration_rate)
    exploration_rate *= 0.9995  # exponential decay
    
end_time =time.time()
result_q = pd.DataFrame(q_s_a, columns =['n','s'], index= states)
result_pi = pd.DataFrame(pi, columns =['n','s'], index= states)
print("Time difference of {} sec".format(end_time- beg_time))
print("-------------")
print(result_pi.T)
print("-------------")
print(result_q.T)
print("-------------");
print("MSE :",((q_s_a-real_solution)**2).mean(), " decaying : ",0.9995, "learning_rate : ", 0.009)
print("\n")
       

Time difference of 23.421976804733276 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0
-------------
        0         10        20        30        40        50        60   70
n -5.47616 -4.699614 -3.790846 -2.670674 -1.887589 -2.058471 -1.000000  0.0
s -5.03484 -4.344880 -3.368675 -3.425124 -1.625501 -1.883980 -1.717377  0.0
-------------
MSE : 0.013233243267497672  decaying :  0.9995 learning_rate :  0.009




###  Policy iteration 2 - TD Control 

In [61]:

for decay in tqdm(exploration_decay)  :
    for lr in learning_rate :
        beg_time =time.time()
        num_ep = 10**5
        q_s_a = q_s_a_init
        pi = pi_50
        exploration_rate = 1

        for epi_i in range(1,num_ep) :
            s_now = 0
            while s_now != 70:
                sample_step = simul_step(pi,s_now, P_normal, P_speed, R_s_a)
                q_s_a = pol_eval_TD(sample_step, q_s_a, alpha = lr)
                pi = pol_imp(pi, q_s_a, epsilon=  exploration_rate)
                s_now = int(sample_step[3])
                exploration_rate *=decay
    
        end_time =time.time()
        result_q = pd.DataFrame(q_s_a, columns =['n','s'], index= states)
        result_pi = pd.DataFrame(pi, columns =['n','s'], index= states)
        print("Time difference of {} sec".format(end_time- beg_time))
        print("-------------")
        print(result_pi.T)
        print("-------------")
        print(result_q.T)
        print("-------------");
        print("MSE :",((q_s_a-real_solution)**2).mean(), " decaying : ",decay, "learning_rate : ", lr)
        print("\n")
       


  0%|                                                                                           | 0/22 [00:00<?, ?it/s]

Time difference of 37.25150775909424 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.284639 -4.464387 -3.623924 -2.673595 -1.653964 -1.771733 -1.000000  0.0
s -5.093668 -4.504751 -3.465008 -2.700838 -1.738410 -1.648088 -1.001889  0.0
-------------
MSE : 0.05883858689132981  decaying :  0.989 learning_rate :  0.005


Time difference of 38.5605890750885 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.291599 -4.490689 -3.611172 -2.647802 -1.630978 -1.776664 -1.000000  0.0
s -5.025738 -4.415729 -3.370768 -2.688467 -1.732553 -1.649007 -1.002441  0.0
-------------
MSE : 0.05991619986730685  decaying :  0.989 learni

  5%|███▋                                                                            | 1/22 [11:22<3:58:45, 682.17s/it]

Time difference of 46.39358687400818 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.634474 -4.689121 -3.638181 -2.635210 -1.625251 -1.928731 -0.999997  0.0
s -5.308713 -4.691197 -3.814310 -2.832787 -1.842296 -1.613517 -1.004912  0.0
-------------
MSE : 0.06947071246625905  decaying :  0.989 learning_rate :  0.019


Time difference of 39.42668128013611 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.249587 -4.494035 -3.618217 -2.655320 -1.639590 -1.787523 -1.000000  0.0
s -4.991627 -4.305466 -3.401205 -2.690604 -1.742983 -1.619628 -1.006255  0.0
-------------
MSE : 0.06090953483259216  decaying :  0.989499999

  9%|███████▎                                                                        | 2/22 [22:46<3:47:45, 683.27s/it]

...

Time difference of 41.6390278339386 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.614099 -4.680898 -3.671223 -2.670008 -1.656126 -1.890015 -1.000000  0.0
s -5.342207 -4.701412 -3.778084 -2.781020 -1.804356 -1.623181 -1.011565  0.0
-------------
MSE : 0.0706993061222711  decaying :  0.9984999999999989 learning_rate :  0.019


Time difference of 36.02609348297119 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.248080 -4.452390 -3.608051 -2.671173 -1.656686 -1.777579 -1.000000  0.0
s -5.082957 -4.491461 -3.396804 -2.687700 -1.721010 -1.643645 -1.000997  0.0
-------------
MSE : 0.060386284173367534  decaying : 

 95%|███████████████████████████████████████████████████████████████████████████▍   | 21/22 [3:40:20<10:07, 607.41s/it]

Time difference of 42.28607130050659 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.621248 -4.666660 -3.667852 -2.663939 -1.662482 -1.956191 -1.000000  0.0
s -5.330409 -4.684181 -3.816021 -2.854561 -1.890352 -1.677778 -1.101427  0.0
-------------
MSE : 0.0607098598182233  decaying :  0.9989999999999989 learning_rate :  0.019


Time difference of 36.681925773620605 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.254325 -4.444317 -3.614700 -2.681708 -1.661962 -1.762528 -1.000000  0.0
s -5.066171 -4.492045 -3.365867 -2.682144 -1.728307 -1.709938 -1.004495  0.0
-------------
MSE : 0.061259885398455685  decaying 

100%|███████████████████████████████████████████████████████████████████████████████| 22/22 [3:50:12<00:00, 627.85s/it]

Time difference of 41.67900347709656 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  0.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50       60   70
n -5.582991 -4.509738 -3.765818 -2.789899 -1.684302 -1.929910 -1.00000  0.0
s -5.182890 -4.644974 -3.508499 -2.801801 -1.830642 -1.656382 -1.31495  0.0
-------------
MSE : 0.03590317820872851  decaying :  0.9994999999999988 learning_rate :  0.019




### 파라미터 실험 

MSE : 0.03590317820872851  decaying :  0.9994999999999988 learning_rate :  0.019

변동성이 너무 큼 

-> 위의 search 결과는 운이 좋아서 그런것으로 보임

In [52]:
beg_time =time.time()
num_ep = 10**5
q_s_a = q_s_a_init
pi = pi_50
exploration_rate = 1

for epi_i in range(1,num_ep) :
            s_now = 0
            while s_now != 70:
                sample_step = simul_step(pi,s_now, P_normal, P_speed, R_s_a)
                q_s_a = pol_eval_TD(sample_step, q_s_a, alpha = 0.019)
                pi = pol_imp(pi, q_s_a, epsilon=  exploration_rate)
                s_now = int(sample_step[3])
                exploration_rate *=0.9994999999999988 
    
end_time =time.time()
result_q = pd.DataFrame(q_s_a, columns =['n','s'], index= states)
result_pi = pd.DataFrame(pi, columns =['n','s'], index= states)
print("Time difference of {} sec".format(end_time- beg_time))
print("-------------")
print(result_pi.T)
print("-------------")
print(result_q.T)
print("-------------");
print("MSE :",((q_s_a-real_solution)**2).mean(), " decaying : ",0.9995, "learning_rate : ", 0.019)
print("\n")
       

Time difference of 43.046430587768555 sec
-------------
    0    10   20   30   40   50   60   70
n  0.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0
s  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
-------------
         0         10        20        30        40        50        60   70
n -5.638351 -4.668637 -3.660703 -2.656543 -1.647700 -1.925164 -1.000000  0.0
s -5.307431 -4.697560 -3.796381 -2.800954 -1.852994 -1.608778 -1.310761  0.0
-------------
MSE : 0.05111189729836377  decaying :  0.9995 learning_rate :  0.019


